<img src="../biospytial.png" alt="Drawing" style="width: 500px;"/>

In [1]:
## Import modules and libraries
%matplotlib inline
import sys
sys.path.append('/apps')
import django
django.setup()
import numpy as np
import matplotlib.pyplot as plt

# Mosquitos, the environment and related species

<img src="mosquito2.jpg" alt="Drawing" style="width: 500px;"/>

## Load Biospytial Modules

In [2]:
## Import modules
from drivers.tree_builder import TreeNeo
from drivers.graph_models import Order, Family, graph, pickNode
from traversals.strategies import sumTrees, UniformRandomSampleForest

## Let´s pick the Mosquito Node
Remember that Mosquitos are all from the Family *Culicidae*
<img src="mosquito3.jpg" alt="taken from: http://www.insectsexplained.com/11Diptera.htm" style="width: 500px;"/>

In [3]:
mosquitos = pickNode(Family,"Culicidae")

In [4]:
mosquitos

<TreeNode type: Family id = 729651 name: Culicidae>

In [5]:
%time cells = list(mosquitos.cells)
neighbours = map(lambda cell : cell.getNeighbours(),cells)
neighbouring_cells = reduce(lambda a,b : a+b ,neighbours)
## Remove repetitions
neighbours = list(set(neighbouring_cells))

CPU times: user 84 ms, sys: 0 ns, total: 84 ms
Wall time: 117 ms


In [6]:
len(neighbours)

39

In [7]:
sample = UniformRandomSampleForest(neighbours,size=5)
%time sample_trees = list(sample)

CPU times: user 25.8 s, sys: 904 ms, total: 26.7 s
Wall time: 29.1 s


#### Wait.. wait..
<img src="clock.png" alt="Drawing" style="width: 100px;"/>
<a href="http://www.youtube.com/watch?feature=player_embedded&v=IkdmOVejUlI
" target="_blank"><img src="http://img.youtube.com/vi/IkdmOVejUlI/0.jpg" 
alt="Waiting music" width="10" height="10" border="10" /></a>

## Explore the nodes !

In [8]:
t1 = sample_trees[0]

In [9]:
t1

<LocalTree Of Life | Root: LUCA - n.count : 53- >

## By Families, Species, Genera, etc

In [10]:
t1.families

[<LocalTree | Family: Plantaginaceae - n.count : 1- | AF: 0.05 >,
 <LocalTree | Family: Aizoaceae - n.count : 2- | AF: 0.05 >,
 <LocalTree | Family: Homalometridae - n.count : 1- | AF: 0.05 >,
 <LocalTree | Family: Heterophyidae - n.count : 1- | AF: 0.05 >,
 <LocalTree | Family: Onagraceae - n.count : 1- | AF: 0.05 >,
 <LocalTree | Family: Gekkonidae - n.count : 1- | AF: 0.05 >,
 <LocalTree | Family: Colubridae - n.count : 1- | AF: 0.05 >,
 <LocalTree | Family: Corytophanidae - n.count : 1- | AF: 0.05 >,
 <LocalTree | Family: Hirundinidae - n.count : 1- | AF: 0.05 >,
 <LocalTree | Family: Tyrannidae - n.count : 1- | AF: 0.05 >,
 <LocalTree | Family: Mimidae - n.count : 1- | AF: 0.05 >,
 <LocalTree | Family: Ciconiidae - n.count : 1- | AF: 0.05 >,
 <LocalTree | Family: Hylidae - n.count : 4- | AF: 0.05 >,
 <LocalTree | Family: Ranidae - n.count : 4- | AF: 0.05 >,
 <LocalTree | Family: Anatidae - n.count : 9- | AF: 0.05 >,
 <LocalTree | Family: Typhaceae - n.count : 1- | AF: 0.05 >,
 <Lo

# Monoid operations

In [11]:
# Let
t1, t2 = sample_trees[:2]

In [12]:
t1.phyla

[<LocalTree | Phylum: Chordata - n.count : 40- | AF: 0.05 >,
 <LocalTree | Phylum: Platyhelminthes - n.count : 2- | AF: 0.05 >,
 <LocalTree | Phylum: Magnoliophyta - n.count : 11- | AF: 0.05 >]

In [13]:
t2.phyla

[<LocalTree | Phylum: Chordata - n.count : 671- | AF: 0.05 >,
 <LocalTree | Phylum: Mollusca - n.count : 2- | AF: 0.05 >,
 <LocalTree | Phylum: Arthropoda - n.count : 574- | AF: 0.05 >,
 <LocalTree | Phylum: Magnoliophyta - n.count : 448- | AF: 0.05 >,
 <LocalTree | Phylum: Pteridophyta - n.count : 9- | AF: 0.05 >]

## Sum

In [14]:
t1 + t2 

INFO Merging Trees


<LocalTree Of Life | Root: LUCA - n.count : 1757- >

### Folding (*Integrate* ) the regional tree

In [15]:
big_t = reduce(lambda a , b : a+b , sample_trees)

INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees


# Tree subselections 

In [16]:
mammals = big_t.to_Animalia.to_Chordata.to_Mammalia.plantTreeNode()

In [17]:
mammals

<LocalTree Of Life | Root: LUCA - n.count : 3- >

# Get (Raster) Environmental Information

In [18]:
table = mammals.associatedData.getEnvironmentalVariablesPoints()

ValueError: need more than 0 values to unpack

In [ ]:
table

### Export to R

In [ ]:
x = table.to_json()
from drivers.tools import redisConnection as rc
r = rc()
r.set('spatstats2017',x)
r.keys()

#### Explore array with numpy

In [ ]:
t1.associatedData.getAssociatedRasterAreaData('MeanTemperature').display_field()

# Rank co-occurrent Nodes

For this we will calculate the frequency of each Node in the big tree, using another list of trees. In this case the sample of trees that we've got.

In [ ]:
big_t.countNodesFrequenciesOnList(list_of_trees=sample_trees)

In [ ]:
big_t.rankLevels()

In [ ]:
big_t.families[:5]

### And for mammals ?

In [ ]:
mammals.countNodesFrequenciesOnList(sample_trees)

In [ ]:
mammals.rankLevels()

In [ ]:
mammals.orders[:5]

# Graph analytics

## Convert the TreeNeo object to a Network instance (networkx)
### The depth_level restricts the depth of the recursion process.

In [ ]:
gt = big_t.toNetworkx(depth_level=6)

In [ ]:
## Plot the Tree
from drivers.tools import plotTree
x = plotTree(big_t)

## Standard Graph analysis
With the **networkx** library

In [ ]:
import networkx as nt
nt.laplacian_matrix(gt)

In [ ]:
sp = nt.laplacian_spectrum(gt)

In [ ]:
sp.shape